In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

product_data = [
(2,"samsung","01-01-1995",11000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-01-2006",15000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(3,"oneplus","01-01-2010",23000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

product_df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         2|     samsung|01-01-1995|  11000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-01-2006|  15000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         3|     oneplus|01-01-2010|  23000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [0]:
window1 = Window.partitionBy('product_id').orderBy('sales_date')

product_df.withColumn('first_val', first('sales').over(window1))\
    .withColumn('last_val', last('sales').over(window1)).show()

+----------+------------+----------+-------+---------+--------+
|product_id|product_name|sales_date|  sales|first_val|last_val|
+----------+------------+----------+-------+---------+--------+
|         1|      iphone|01-01-2006|  15000|    15000|   15000|
|         1|      iphone|01-02-2023|1300000|    15000| 1300000|
|         1|      iphone|01-03-2023|1600000|    15000| 1600000|
|         1|      iphone|01-04-2023|1700000|    15000| 1700000|
|         1|      iphone|01-05-2023|1200000|    15000| 1200000|
|         1|      iphone|01-06-2023|1100000|    15000| 1100000|
|         2|     samsung|01-01-1995|  11000|    11000|   11000|
|         2|     samsung|01-02-2023|1120000|    11000| 1120000|
|         2|     samsung|01-03-2023|1080000|    11000| 1080000|
|         2|     samsung|01-04-2023|1800000|    11000| 1800000|
|         2|     samsung|01-05-2023| 980000|    11000|  980000|
|         2|     samsung|01-06-2023|1100000|    11000| 1100000|
|         3|     oneplus|01-01-2010|  23

In [0]:
window1 = Window.partitionBy('product_id').orderBy('sales_date')

product_df.withColumn('first_val', first('sales').over(window1))\
    .withColumn('last_val', last('sales').over(window1)).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Window [product_id#3980L, product_name#3981, sales_date#3982, sales#3983L, first(sales#3983L, false) windowspecdefinition(product_id#3980L, sales_date#3982 ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS first_val#4108L, last(sales#3983L, false) windowspecdefinition(product_id#3980L, sales_date#3982 ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS last_val#4115L], [product_id#3980L], [sales_date#3982 ASC NULLS FIRST]
   +- Sort [product_id#3980L ASC NULLS FIRST, sales_date#3982 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(product_id#3980L, 200), ENSURE_REQUIREMENTS, [plan_id=7344]
         +- Scan ExistingRDD[product_id#3980L,product_name#3981,sales_date#3982,sales#3983L]




In [0]:
window1 = Window.partitionBy('product_id').orderBy('sales_date').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

f_l_df = product_df.withColumn('first_val', first('sales').over(window1))\
    .withColumn('last_val', last('sales').over(window1))

f_l_df.show()

+----------+------------+----------+-------+---------+--------+
|product_id|product_name|sales_date|  sales|first_val|last_val|
+----------+------------+----------+-------+---------+--------+
|         1|      iphone|01-01-2006|  15000|    15000| 1100000|
|         1|      iphone|01-02-2023|1300000|    15000| 1100000|
|         1|      iphone|01-03-2023|1600000|    15000| 1100000|
|         1|      iphone|01-04-2023|1700000|    15000| 1100000|
|         1|      iphone|01-05-2023|1200000|    15000| 1100000|
|         1|      iphone|01-06-2023|1100000|    15000| 1100000|
|         2|     samsung|01-01-1995|  11000|    11000| 1100000|
|         2|     samsung|01-02-2023|1120000|    11000| 1100000|
|         2|     samsung|01-03-2023|1080000|    11000| 1100000|
|         2|     samsung|01-04-2023|1800000|    11000| 1100000|
|         2|     samsung|01-05-2023| 980000|    11000| 1100000|
|         2|     samsung|01-06-2023|1100000|    11000| 1100000|
|         3|     oneplus|01-01-2010|  23

In [0]:
# diff in sales for each product from their 1st month sales to latest sales

f_l_df.withColumn('diff_in_sales', (col('first_val')-col('last_val'))).show()

+----------+------------+----------+-------+---------+--------+-------------+
|product_id|product_name|sales_date|  sales|first_val|last_val|diff_in_sales|
+----------+------------+----------+-------+---------+--------+-------------+
|         1|      iphone|01-01-2006|  15000|    15000| 1100000|     -1085000|
|         1|      iphone|01-02-2023|1300000|    15000| 1100000|     -1085000|
|         1|      iphone|01-03-2023|1600000|    15000| 1100000|     -1085000|
|         1|      iphone|01-04-2023|1700000|    15000| 1100000|     -1085000|
|         1|      iphone|01-05-2023|1200000|    15000| 1100000|     -1085000|
|         1|      iphone|01-06-2023|1100000|    15000| 1100000|     -1085000|
|         2|     samsung|01-01-1995|  11000|    11000| 1100000|     -1089000|
|         2|     samsung|01-02-2023|1120000|    11000| 1100000|     -1089000|
|         2|     samsung|01-03-2023|1080000|    11000| 1100000|     -1089000|
|         2|     samsung|01-04-2023|1800000|    11000| 1100000| 

In [0]:
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

emp_df.show()

+---+------+----------+-----+
| id|  name|      date| time|
+---+------+----------+-----+
|  1|manish|11-07-2023|10:20|
|  1|manish|11-07-2023|11:20|
|  2|rajesh|11-07-2023|11:20|
|  1|manish|11-07-2023|11:50|
|  2|rajesh|11-07-2023|13:20|
|  1|manish|11-07-2023|19:20|
|  2|rajesh|11-07-2023|17:20|
|  1|manish|12-07-2023|10:32|
|  1|manish|12-07-2023|12:20|
|  3|vikash|12-07-2023|09:12|
|  1|manish|12-07-2023|16:23|
|  3|vikash|12-07-2023|18:08|
+---+------+----------+-----+



In [0]:
## send email to employee who have not completed 8 hours

emp_t_df = emp_df.withColumn('timestamp', from_unixtime(unix_timestamp(expr('CONCAT(date, " ", time)'), 'dd-MM-yyyy HH:mm')))

emp_t_df.show()

+---+------+----------+-----+-------------------+
| id|  name|      date| time|          timestamp|
+---+------+----------+-----+-------------------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|
|  2|rajesh|11-07-2023|11:20|2023-07-11 11:20:00|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|
|  2|rajesh|11-07-2023|13:20|2023-07-11 13:20:00|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|
|  2|rajesh|11-07-2023|17:20|2023-07-11 17:20:00|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|
|  3|vikash|12-07-2023|09:12|2023-07-12 09:12:00|
|  1|manish|12-07-2023|16:23|2023-07-12 16:23:00|
|  3|vikash|12-07-2023|18:08|2023-07-12 18:08:00|
+---+------+----------+-----+-------------------+



In [0]:
window1 = Window.partitionBy('id', 'date').orderBy('date').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

emp_t_df.withColumn("login", first('timestamp').over(window1))\
    .withColumn("logout", last('timestamp').over(window1))\
        .withColumn('total_time', (col('logout')-col('login'))).printSchema()


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- login: string (nullable = true)
 |-- logout: string (nullable = true)
 |-- total_time: double (nullable = true)



In [0]:
window1 = Window.partitionBy('id', 'date').orderBy('date').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

log_df = emp_t_df.withColumn("login", first('timestamp').over(window1))\
    .withColumn("logout", last('timestamp').over(window1))\
        .withColumn('total_time', (col('logout')-col('login')))

log_df.show()


+---+------+----------+-----+-------------------+-------------------+-------------------+----------+
| id|  name|      date| time|          timestamp|              login|             logout|total_time|
+---+------+----------+-----+-------------------+-------------------+-------------------+----------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|      null|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|      null|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|2023-07-11 10:20:00|2023-07-11 19:20:00|      null|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|      null|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|2023-07-12 10:32:00|2023-07-12 16:23:00|      null|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|2023-07-12 10:32:00|2023-07-12 16:23:00|      null|
|  1|manish|12-07-2023|16:23|2023-07-12 16:23:00|2023-07-12 10:32:00|2023-07-12 16:23:00|  

In [0]:
window1 = Window.partitionBy('id', 'date').orderBy('date').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

total_df = emp_t_df.withColumn("login", first('timestamp').over(window1))\
    .withColumn("logout", last('timestamp').over(window1))\
    .withColumn('login', to_timestamp("login", 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn('logout', to_timestamp("logout", 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn('total_time', (col('logout')-col('login')))
    
total_df.printSchema()


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- login: timestamp (nullable = true)
 |-- logout: timestamp (nullable = true)
 |-- total_time: interval day to second (nullable = true)



In [0]:
window1 = Window.partitionBy('id', 'date').orderBy('date').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

total_df = emp_t_df.withColumn("login", first('timestamp').over(window1))\
    .withColumn("logout", last('timestamp').over(window1))\
    .withColumn('login', to_timestamp("login", 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn('logout', to_timestamp("logout", 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn('total_time', (col('logout')-col('login')))
    
total_df.show()


+---+------+----------+-----+-------------------+-------------------+-------------------+--------------------+
| id|  name|      date| time|          timestamp|              login|             logout|          total_time|
+---+------+----------+-----+-------------------+-------------------+-------------------+--------------------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|2023-07-12 10:32:00|2023-07-12 16:23:00|INTERVAL '0 05:51...|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|2023-07-12 10:32:00|2023-07-12 16:23:00|INTERVAL '0 05:51...|
|

In [0]:
total_df.select('id', 'name', 'date', 'total_time').distinct().show()    # send email to those

+---+------+----------+--------------------+
| id|  name|      date|          total_time|
+---+------+----------+--------------------+
|  1|manish|11-07-2023|INTERVAL '0 09:00...|
|  1|manish|12-07-2023|INTERVAL '0 05:51...|
|  2|rajesh|11-07-2023|INTERVAL '0 06:00...|
|  3|vikash|12-07-2023|INTERVAL '0 08:56...|
+---+------+----------+--------------------+



In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema) 
product_df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         1|      iphone|01-01-2023|1500000|
|         2|     samsung|01-01-2023|1100000|
|         3|     oneplus|01-01-2023|1100000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [0]:
## performance of sales based on last 3 months avg [drop 1st 2 records for each product then divide by 3]

wind1 = Window.partitionBy('product_id').orderBy('sales_date').rowsBetween(-2, 0)

product_df.withColumn("total_sum", sum('sales').over(wind1)).show()

+----------+------------+----------+-------+---------+
|product_id|product_name|sales_date|  sales|total_sum|
+----------+------------+----------+-------+---------+
|         1|      iphone|01-01-2023|1500000|  1500000|
|         1|      iphone|01-02-2023|1300000|  2800000|
|         1|      iphone|01-03-2023|1600000|  4400000|
|         1|      iphone|01-04-2023|1700000|  4600000|
|         1|      iphone|01-05-2023|1200000|  4500000|
|         1|      iphone|01-06-2023|1100000|  4000000|
|         2|     samsung|01-01-2023|1100000|  1100000|
|         2|     samsung|01-02-2023|1120000|  2220000|
|         2|     samsung|01-03-2023|1080000|  3300000|
|         2|     samsung|01-04-2023|1800000|  4000000|
|         2|     samsung|01-05-2023| 980000|  3860000|
|         2|     samsung|01-06-2023|1100000|  3880000|
|         3|     oneplus|01-01-2023|1100000|  1100000|
|         3|     oneplus|01-02-2023|1120000|  2220000|
|         3|     oneplus|01-03-2023|1160000|  3380000|
|         